# Generate Detector Evaluation Trajectory

This notebook generates a pre-computed trajectory for evaluating pluggable fault detectors (40-series notebooks).

**Scenario**:
1. Start with 8 hours of normal operation
2. For each fault (1, 2, 4-8, 10-14, 16-20):
   - Turn fault ON for 1 hour
   - Turn fault OFF for 1 hour (recovery)
3. Total simulation: 8 + (17 faults × 2 hours) = 42 hours

**Output Files**:
- `data/detector_trajectory.csv` - Full trajectory with measurements and ground truth
- `data/detector_trajectory_quick.csv` - Quick mode (shorter simulation)

## Configuration

In [1]:
import os
import time
import numpy as np
import pandas as pd
from pathlib import Path
from tep import TEPSimulator

# =============================================================================
# QUICK MODE: Set to True for fast testing with shorter simulation
# =============================================================================
QUICK_MODE = os.environ.get('QUICK_MODE', 'False').lower() in ('true', '1', 'yes')

# Paths
DATA_DIR = Path('../data')
DATA_DIR.mkdir(exist_ok=True)

# Simulation parameters
RECORD_INTERVAL = 180  # 3 minutes = 180 seconds
SEED = 2000            # Different seed from other datasets

# Fault classes (excluding 3, 9, 15 which are undetectable)
FAULT_CLASSES_FULL = [1, 2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20]

if QUICK_MODE:
    # Quick mode: fewer faults, shorter periods
    NORMAL_HOURS = 2           # 2 hours normal
    FAULT_ON_HOURS = 0.5       # 30 min fault
    FAULT_OFF_HOURS = 0.5      # 30 min recovery
    FAULT_CLASSES = [1, 2, 5, 10, 17]  # Only 5 representative faults
    FILE_SUFFIX = '_quick'
    print("="*60)
    print("⚡ QUICK MODE - Generating short detector trajectory")
    print("="*60)
else:
    # Full mode: all faults, standard periods
    NORMAL_HOURS = 8           # 8 hours normal
    FAULT_ON_HOURS = 1         # 1 hour fault
    FAULT_OFF_HOURS = 1        # 1 hour recovery
    FAULT_CLASSES = FAULT_CLASSES_FULL
    FILE_SUFFIX = ''
    print("="*60)
    print("Detector Trajectory Generation (FULL MODE)")
    print("="*60)

# Calculate total duration
TOTAL_HOURS = NORMAL_HOURS + len(FAULT_CLASSES) * (FAULT_ON_HOURS + FAULT_OFF_HOURS)
SAMPLES_PER_HOUR = 3600 // RECORD_INTERVAL  # 20 samples per hour

print(f"Normal period: {NORMAL_HOURS} hours")
print(f"Fault ON period: {FAULT_ON_HOURS} hour(s)")
print(f"Fault OFF period: {FAULT_OFF_HOURS} hour(s)")
print(f"Number of faults: {len(FAULT_CLASSES)}")
print(f"Total simulation: {TOTAL_HOURS} hours")
print(f"Expected samples: ~{int(TOTAL_HOURS * SAMPLES_PER_HOUR)}")
print("="*60)

⚡ QUICK MODE - Generating short detector trajectory
Normal period: 2 hours
Fault ON period: 0.5 hour(s)
Fault OFF period: 0.5 hour(s)
Number of faults: 5
Total simulation: 7.0 hours
Expected samples: ~140


## Build Fault Injection Scenario

In [2]:
print("Building fault injection scenario...")

# Build ground truth schedule
# List of (start_time_hours, end_time_hours, fault_class)
ground_truth_schedule = []

# Add initial normal period
ground_truth_schedule.append((0, NORMAL_HOURS, 0))

current_time = NORMAL_HOURS

print(f"\nFault Schedule:")
print(f"{'Time (h)':<15} {'Duration':<12} {'Fault':<8} {'Status'}")
print("-" * 50)
print(f"0.0 - {NORMAL_HOURS:.1f}       {NORMAL_HOURS}h           0        Normal")

for fault_id in FAULT_CLASSES:
    fault_start = current_time
    fault_end = fault_start + FAULT_ON_HOURS
    recovery_end = fault_end + FAULT_OFF_HOURS
    
    # Ground truth for fault period and recovery
    ground_truth_schedule.append((fault_start, fault_end, fault_id))
    ground_truth_schedule.append((fault_end, recovery_end, 0))  # Recovery = normal
    
    print(f"{fault_start:.1f} - {fault_end:.1f}        {FAULT_ON_HOURS}h           {fault_id:<8} Fault ON")
    print(f"{fault_end:.1f} - {recovery_end:.1f}        {FAULT_OFF_HOURS}h           0        Recovery")
    
    current_time = recovery_end

print("-" * 50)
print(f"Total: {TOTAL_HOURS} hours")

Building fault injection scenario...

Fault Schedule:
Time (h)        Duration     Fault    Status
--------------------------------------------------
0.0 - 2.0       2h           0        Normal
2.0 - 2.5        0.5h           1        Fault ON
2.5 - 3.0        0.5h           0        Recovery
3.0 - 3.5        0.5h           2        Fault ON
3.5 - 4.0        0.5h           0        Recovery
4.0 - 4.5        0.5h           5        Fault ON
4.5 - 5.0        0.5h           0        Recovery
5.0 - 5.5        0.5h           10       Fault ON
5.5 - 6.0        0.5h           0        Recovery
6.0 - 6.5        0.5h           17       Fault ON
6.5 - 7.0        0.5h           0        Recovery
--------------------------------------------------
Total: 7.0 hours


## Run Simulation

In [3]:
print("\nRunning TEP simulation segments...")
sim_start = time.time()

np.random.seed(SEED)

# Since tep-sim only supports single fault injection per simulation,
# we run separate simulations for each segment and concatenate results.
# Each segment starts fresh with initialized simulator.

all_times = []
all_measurements = []
all_manipulated_vars = []
all_fault_labels = []

cumulative_time = 0.0
segment_count = 0

# Build list of simulation segments
# Each segment: (duration_hours, fault_id or None, label_for_segment)
segments = []

# Initial normal period
segments.append((NORMAL_HOURS, None, 0))

# Fault periods
for fault_id in FAULT_CLASSES:
    # Fault ON period - inject fault at t=0 of this segment
    segments.append((FAULT_ON_HOURS, fault_id, fault_id))
    # Recovery period - normal operation
    segments.append((FAULT_OFF_HOURS, None, 0))

print(f"Total segments to simulate: {len(segments)}")
print("-" * 50)

for seg_idx, (duration, fault_id, label) in enumerate(segments):
    segment_count += 1
    
    # Initialize fresh simulator for each segment
    sim = TEPSimulator()
    sim.initialize()
    
    # Set up disturbances
    if fault_id is not None:
        # Inject fault at start of segment (t=0)
        disturbances = {fault_id: (0.0, 1)}
        seg_desc = f"Fault {fault_id} ON"
    else:
        disturbances = None
        seg_desc = "Normal"
    
    # Run segment
    result = sim.simulate(
        duration_hours=duration,
        disturbances=disturbances,
        record_interval=RECORD_INTERVAL
    )
    
    if result.shutdown:
        print(f"  WARNING: Segment {seg_idx+1} shutdown at {cumulative_time + result.time[-1]:.2f}h")
    
    # Adjust times to be cumulative
    segment_times = result.time + cumulative_time
    
    # Store results
    all_times.extend(segment_times.tolist())
    all_measurements.append(result.measurements)
    all_manipulated_vars.append(result.manipulated_vars)
    all_fault_labels.extend([label] * len(result.time))
    
    print(f"  Segment {seg_idx+1:2d}/{len(segments)}: {seg_desc:<15} "
          f"t={cumulative_time:.1f}-{cumulative_time+duration:.1f}h, "
          f"{len(result.time)} samples")
    
    cumulative_time += duration

# Concatenate all results
all_times = np.array(all_times)
all_measurements = np.vstack(all_measurements)
all_manipulated_vars = np.vstack(all_manipulated_vars)
all_fault_labels = np.array(all_fault_labels)

sim_time = time.time() - sim_start
print("-" * 50)
print(f"Simulation complete in {sim_time:.2f}s")
print(f"  Total samples: {len(all_times)}")
print(f"  Measurements shape: {all_measurements.shape}")
print(f"  Manipulated vars shape: {all_manipulated_vars.shape}")


Running TEP simulation segments...
Total segments to simulate: 11
--------------------------------------------------


  Segment  1/11: Normal          t=0.0-2.0h, 41 samples


  Segment  2/11: Fault 1 ON      t=2.0-2.5h, 11 samples


  Segment  3/11: Normal          t=2.5-3.0h, 11 samples


  Segment  4/11: Fault 2 ON      t=3.0-3.5h, 11 samples


...

  Segment  7/11: Normal          t=4.5-5.0h, 11 samples


  Segment  8/11: Fault 10 ON     t=5.0-5.5h, 11 samples


  Segment  9/11: Normal          t=5.5-6.0h, 11 samples


  Segment 10/11: Fault 17 ON     t=6.0-6.5h, 11 samples


  Segment 11/11: Normal          t=6.5-7.0h, 11 samples
--------------------------------------------------
Simulation complete in 3.74s
  Total samples: 151
  Measurements shape: (151, 41)
  Manipulated vars shape: (151, 12)


## Build DataFrame with Ground Truth

In [4]:
print("\nBuilding DataFrame...")

n_samples = len(all_times)

# Build data dictionary
data = {
    'step': list(range(n_samples)),
    'time_hours': all_times.tolist(),
}

# Add XMEAS columns (41 measurements)
for i in range(41):
    data[f'xmeas_{i+1}'] = all_measurements[:, i]

# Add XMV columns (11 manipulated variables)
for i in range(11):
    data[f'xmv_{i+1}'] = all_manipulated_vars[:, i]

# Create DataFrame
df = pd.DataFrame(data)

# Add ground truth labels (already computed during simulation)
df['faultNumber'] = all_fault_labels

# Add metadata
df['origin'] = 'detector_trajectory'

print(f"DataFrame created: {df.shape}")
print(f"\nGround truth distribution:")
print(df['faultNumber'].value_counts().sort_index())


Building DataFrame...
DataFrame created: (151, 56)

Ground truth distribution:
faultNumber
0     96
1     11
2     11
5     11
10    11
17    11
Name: count, dtype: int64


## Verify Ground Truth Alignment

In [5]:
print("\nVerifying ground truth alignment...")

# Check transitions
transitions = df[df['faultNumber'].diff() != 0][['step', 'time_hours', 'faultNumber']]
print(f"\nFault transitions detected: {len(transitions)}")
print(transitions.head(20).to_string())

if len(transitions) > 20:
    print(f"... and {len(transitions) - 20} more transitions")

# Verify samples per class
print(f"\nSamples per fault class:")
for fault_id in [0] + list(FAULT_CLASSES):
    count = (df['faultNumber'] == fault_id).sum()
    expected = NORMAL_HOURS * SAMPLES_PER_HOUR if fault_id == 0 else FAULT_ON_HOURS * SAMPLES_PER_HOUR
    # Normal class also includes recovery periods
    if fault_id == 0:
        expected = (NORMAL_HOURS + len(FAULT_CLASSES) * FAULT_OFF_HOURS) * SAMPLES_PER_HOUR
    status = "✓" if abs(count - expected) <= 2 else "?"
    print(f"  Fault {fault_id:2d}: {count:4d} samples (expected ~{int(expected)}) {status}")


Verifying ground truth alignment...

Fault transitions detected: 11
     step  time_hours  faultNumber
0       0         0.0            0
41     41         2.0            1
52     52         2.5            0
63     63         3.0            2
74     74         3.5            0
85     85         4.0            5
96     96         4.5            0
107   107         5.0           10
118   118         5.5            0
129   129         6.0           17
140   140         6.5            0

Samples per fault class:
  Fault  0:   96 samples (expected ~90) ?
  Fault  1:   11 samples (expected ~10) ✓
  Fault  2:   11 samples (expected ~10) ✓
  Fault  5:   11 samples (expected ~10) ✓
  Fault 10:   11 samples (expected ~10) ✓
  Fault 17:   11 samples (expected ~10) ✓


## Save Trajectory

In [6]:
print("\nSaving trajectory...")

# Save main trajectory file
output_file = DATA_DIR / f'detector_trajectory{FILE_SUFFIX}.csv'
df.to_csv(output_file, index=False)
file_size_mb = output_file.stat().st_size / 1e6
print(f"✓ Saved to {output_file} ({file_size_mb:.1f} MB)")

# Save ground truth schedule as JSON for reference
import json

schedule_info = {
    'quick_mode': QUICK_MODE,
    'total_hours': TOTAL_HOURS,
    'total_samples': n_samples,
    'normal_hours': NORMAL_HOURS,
    'fault_on_hours': FAULT_ON_HOURS,
    'fault_off_hours': FAULT_OFF_HOURS,
    'record_interval_seconds': RECORD_INTERVAL,
    'fault_classes': FAULT_CLASSES,
    'ground_truth_schedule': [
        {'start_hours': s, 'end_hours': e, 'fault_class': f}
        for s, e, f in ground_truth_schedule
    ],
    'seed': SEED
}

schedule_file = DATA_DIR / f'detector_trajectory{FILE_SUFFIX}_info.json'
with open(schedule_file, 'w') as f:
    json.dump(schedule_info, f, indent=2)
print(f"✓ Saved schedule info to {schedule_file}")


Saving trajectory...
✓ Saved to ../data/detector_trajectory_quick.csv (0.2 MB)
✓ Saved schedule info to ../data/detector_trajectory_quick_info.json


In [7]:
print("\n" + "="*60)
print("Detector Trajectory Generation Complete!")
if QUICK_MODE:
    print("(Quick mode)")
print("="*60)
print(f"\nSummary:")
print(f"  Duration: {TOTAL_HOURS} hours")
print(f"  Samples: {n_samples:,}")
print(f"  Faults: {len(FAULT_CLASSES)} types")
print(f"  Normal samples: {(df['faultNumber'] == 0).sum():,}")
print(f"  Fault samples: {(df['faultNumber'] != 0).sum():,}")
print(f"\nOutput files:")
print(f"  - {output_file}")
print(f"  - {schedule_file}")
print("="*60)


Detector Trajectory Generation Complete!
(Quick mode)

Summary:
  Duration: 7.0 hours
  Samples: 151
  Faults: 5 types
  Normal samples: 96
  Fault samples: 55

Output files:
  - ../data/detector_trajectory_quick.csv
  - ../data/detector_trajectory_quick_info.json
